## VGGNet
VGGNet是通过简单堆叠卷积构建网络的巅峰之作。

VGGNet是由牛津大学的Visual Geometry Group提出的一种结构，该架构获得了2014年ILSVRC分类任务的亚军以及定位任务的冠军。它的实质是使用了比AlexNet更小的卷积核与更深的网络。在这里，我们将试图实现VGGNet16。

VGGNet16的架构为：输入层、卷积层、卷积层、池化层、卷积层、卷积层、池化层、卷积层、卷积层、卷积层、池化层、卷积层、卷积层、卷积层、池化层、卷积层、卷积层、卷积层、池化层、全连接层、全连接层、全连接层、输出层。

### 1. 导入必要模块

In [1]:
import time as time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

### 2. 引入数据集

在这里，我们直接使用tensorflow中自带的数据集。

In [2]:
# These variables are all in type of numpy.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 3. 数据预处理

将28\*28的图片填充到32\*32的规模，以便于进行输入。将图片变为3维，以便于神经网络的训练。同时，将分类变为one-hot编码，以便于后续在神经网络训练中可以使用categorical cross-entropy损失函数。

In [3]:
train_images_32 = np.zeros((60000, 32, 32), dtype=train_images.dtype)
test_images_32 = np.zeros((10000, 32, 32), dtype=test_images.dtype)

start_row = (32 - 28) // 2
start_col = (32 - 28) // 2
for i in range(60000):
  train_images_32[i][start_row:start_row+28, start_col:start_col+28] = train_images[i]
for i in range(10000):
  test_images_32[i][start_row:start_row+28, start_col:start_col+28] = test_images[i]

train_images_32 = train_images_32.reshape((60000, 32, 32, 1)).astype('float32') / 255
test_images_32 = test_images_32.reshape((10000, 32, 32, 1)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_images_32.shape)
print(test_images_32.shape)
print(train_labels.shape)
print(test_labels.shape)

(60000, 32, 32, 1)
(10000, 32, 32, 1)
(60000, 10)
(10000, 10)


### 4. 搭建神经网络

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

model.add(layers.Flatten())
# model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### 5. 编译模型

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### 6. 训练模型

In [6]:
start_time = time.time()
model.fit(train_images_32, train_labels, epochs=5, batch_size=64, validation_split=0.2)
end_time = time.time()
print("Running time:", end_time - start_time, "seconds")

Epoch 1/5
750/750 [==============================] - 33s 28ms/step - loss: 2.3015 - accuracy: 0.1132 - val_loss: 2.3021 - val_accuracy: 0.1060
Epoch 2/5
750/750 [==============================] - 21s 28ms/step - loss: 2.3013 - accuracy: 0.1140 - val_loss: 2.3019 - val_accuracy: 0.1060
Epoch 3/5
750/750 [==============================] - 19s 26ms/step - loss: 2.3012 - accuracy: 0.1140 - val_loss: 2.3022 - val_accuracy: 0.1060
Epoch 4/5
750/750 [==============================] - 19s 26ms/step - loss: 2.3012 - accuracy: 0.1140 - val_loss: 2.3020 - val_accuracy: 0.1060
Epoch 5/5
750/750 [==============================] - 19s 26ms/step - loss: 2.3011 - accuracy: 0.1140 - val_loss: 2.3022 - val_accuracy: 0.1060
Running time: 143.9748764038086 seconds


### 7. 模型用于测试集

In [ ]:
test_loss, test_acc = model.evaluate(test_images_32, test_labels)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 2s 7ms/step - loss: 0.0433 - accuracy: 0.9905
Test accuracy: 0.9904999732971191
